In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm
import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730093977.888668      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1028 05:39:37.896734937      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1028 05:39:37.896749473      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1028 05:39:37.896752822      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1028 05:39:37.896755181      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1028 05:39:37.896757565      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730093994.950855      13 service.cc:145] XLA service 0x5d242a1d3510 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730093994.950907      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730093994.950912      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730093994.950921      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730093994.950924      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730093994.950927      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730093994.950929      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730093994.950932      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730093994.950935      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [5]:
Test = True

if Test:
    num_jobs = 90
else:
    num_jobs = 4

config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [6]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['train_parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

len(final_id_list)

996

In [12]:
# Calculate the approximate size of each smaller list
chunk_size = len(final_id_list) // 90
remainder = len(final_id_list) % 90

# Create the 90 lists
result = []
start = 0

for i in range(90):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(final_id_list[start:end])
    start = end

# Check the result
for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")

List 1 (length 12): ['0fbb3ca5', 'f4d2f5af', 'a8c1aab5', '22c72c4e', 'fecc07d6', '73ffa609', 'b802aec3', '9fa5cdfd', 'aecc7e74', '913bea9f', 'd661ddf7', '11f9b26c']
List 2 (length 12): ['a08ebf6e', '4a9f92ca', 'c09d41cd', '43f5d3c5', '34ec5de4', '05bbed1b', '35198c2d', 'e328a032', '2a88cbe9', 'e21255e2', 'e3f71f7a', '57f6fa6c']
List 3 (length 12): ['fd0d9bca', '24ab10c6', '17c1ae23', '38d53b92', '224fc39a', 'cefdb7fe', 'b23b9220', '35d2aa41', '7a6d26a2', 'd859115e', 'a0182b17', 'b3f5c066']
List 4 (length 12): ['9e171735', '428c37ea', '432aff11', '6fb28c2a', 'de742c5c', 'bdcc7d10', '7b543871', '7265f1e5', '0f6b71c4', '119e181e', '5d885b20', 'd8037389']
List 5 (length 12): ['eb93a46a', '687c85e7', '764d8b42', 'db23fbe4', 'f584742f', '75b53bd4', '0c98f822', '5ddbac80', 'b41cee02', '0c4e2d68', '07048067', 'ad0bbf35']
List 6 (length 12): ['df556fd2', '11be5d2e', '1f2164a4', '6b6467f4', '7e46b693', 'ed711416', '06eb8adb', '47350ac6', 'b7a61f8d', '233b9425', '4d61a44e', 'aae9800e']
List 7 (le

In [13]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [36]:
train_data_sample = train_data.filter(pl.col('id')=='00115b9f')

sample_joined_features = dd.sql("select t1.*, t2.* exclude(id) \
from train_data_sample t1 join parquet_sample_df t2 on t1.id = t2.id").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
sample_joined_features.head(10)

id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,weekday,quarter,relative_date_PCIAT,hour_of_day,min_of_day,sec_of_day
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64,u32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,f32,i32,i32,i32
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43329,0.792144,0.170486,-0.413108,0.063103,-16.771301,0.0,0.0,3824.0,6,3,85.0,5,46,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43328,0.801953,0.501589,-0.040937,0.045489,-2.113776,0.0,0.0,3824.0,6,3,85.0,8,53,20
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43327,0.38737,0.793151,-0.402214,0.069961,-24.097908,0.0,0.0,3824.0,6,3,85.0,12,0,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43326,0.04873,-0.076725,-0.953776,0.056839,-85.352219,0.0,0.0,3824.0,6,3,85.0,15,6,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43325,-0.008333,-0.02362,-1.006901,0.007224,-88.595741,0.0,0.0,3824.0,6,3,85.0,4,26,40
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43324,-0.008542,-0.023385,-1.006797,0.007117,-88.595741,0.0,0.0,3824.0,6,3,85.0,7,33,20
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1,43323,-0.005859,-0.024089,-1.007552,0.013274,-88.570198,0.0,0.0,3824.0,6,3,85.0,10,40,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292

In [38]:
sample_joined_features.shape

(43330, 97)

In [31]:
df_initializing_id = final_id_list.pop()

parquet_file_name = "id={0}/*.parquet".format(df_initializing_id)
first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name

first_parquet_df = pl.read_parquet(first_parquet_file)
first_parquet_df = first_parquet_df.drop(['id'])
            
first_parquet_df = first_parquet_df.with_columns([
    pl.lit(id).alias("id"),
    pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day"),
    pl.col("time_of_day").map_elements(get_mins, return_dtype=pl.Int32).alias("min_of_day"),
    pl.col("time_of_day").map_elements(get_secs, return_dtype=pl.Int32).alias("sec_of_day")
])

first_parquet_df = first_parquet_df.drop(['time_of_day'])

first_parquet_df.head(100)

step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,weekday,quarter,relative_date_PCIAT,id,hour_of_day,min_of_day,sec_of_day
u32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,f32,str,i32,i32,i32
0,-0.4205,0.371837,-0.434597,0.065384,-25.429186,0.0,8.0,4167.0,4,3,0.0,"""90161e10""",21,20,0
1,-0.273933,0.548216,-0.249921,0.080327,-16.888573,0.0,6.0,4167.0,4,3,0.0,"""90161e10""",18,13,20
2,0.082017,-0.388388,-0.617869,0.085136,-39.957638,0.0,9.6,4167.0,4,3,0.0,"""90161e10""",15,6,40
3,-0.088559,-0.66655,-0.730163,0.023409,-47.634476,0.0,50.400002,4167.0,4,3,0.0,"""90161e10""",12,0,0
4,0.327806,0.170603,-0.668739,0.081186,-51.526844,0.0,91.199997,4167.0,4,3,0.0,"""90161e10""",8,53,20
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
95,-0.432895,-0.614505,-0.029646,0.02684,-2.270243,0.0,18.296297,4170.0,4,3,0.0,"""90161e10""",13,46,40
96,-0.724325,0.513257,-0.375876,0.024603,-23.014355,0.0,19.0,4170.0,4,3,0.0,"""90161e10""",10,40,0
97,-0.870437,0.412427,-0.273329,0.015276,-15.925615,0.0,1.5,4170.0,4,3,0.0,"""90161e10""",7,33,20


In [1]:
#first_parquet_df.shape

dd.sql("select id, count(1) from first_parquet_df group by id").pl()

NameError: name 'dd' is not defined

In [ ]:
with tpu_strategy.scope():
    for id in final_id_list:
        # print(id)
        parquet_file_name = "id={0}/*.parquet".format(id)
        parquet_file_path = config_dict['train_parquet_root_path']+parquet_file_name
        parquet_df = pl.read_parquet(parquet_file_path)
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day"),
            pl.col("time_of_day").map_elements(get_mins, return_dtype=pl.Int32).alias("min_of_day"),
            pl.col("time_of_day").map_elements(get_secs, return_dtype=pl.Int32).alias("sec_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        
        try:
            first_parquet_df = pl.concat([first_parquet_df, parquet_df])
        except:
            print(id)

In [22]:
def unify_parquet_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()

    parquet_file_name = "id={0}/*.parquet".format(df_initializing_id)
    first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name

    first_parquet_df = pl.read_parquet(first_parquet_file)
    first_parquet_df = first_parquet_df.drop(['id'])

    first_parquet_df = first_parquet_df.with_columns([
        pl.lit(df_initializing_id).alias("id"),
        pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day"),
        pl.col("time_of_day").map_elements(get_mins, return_dtype=pl.Int32).alias("min_of_day"),
        pl.col("time_of_day").map_elements(get_secs, return_dtype=pl.Int32).alias("sec_of_day")
    ])

    first_parquet_df = first_parquet_df.drop(['time_of_day'])
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/*.parquet".format(id)
        parquet_file_path = config_dict['train_parquet_root_path']+parquet_file_name
        parquet_df = pl.read_parquet(parquet_file_path)
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day"),
            pl.col("time_of_day").map_elements(get_mins, return_dtype=pl.Int32).alias("min_of_day"),
            pl.col("time_of_day").map_elements(get_secs, return_dtype=pl.Int32).alias("sec_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        
        try:
            first_parquet_df = pl.concat([first_parquet_df, parquet_df])
        except:
            print(id)
    return first_parquet_df

In [23]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_parquet_files)(id_list) for id_list in tqdm(result)))

100%|██████████| 10/10 [01:26<00:00,  8.69s/it]


In [27]:
len(parts_of_parquet)

90

In [28]:
for f, pq_file in enumerate(parts_of_parquet):
    pq_file.write_parquet(f"{pq_file}_{f}")

OSError: File name too long (os error 36)